In [8]:
import os
import io
import sys
# Beispiel: setze Projekt-Root manuell (Pfad anpassen)
project_root = os.path.abspath('..')  # oder '..' oder '../..' je nach Notebook-Ordner

import json
import joblib
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import pandas as pd
from IPython import display  # WICHTIG für Jupyter Live-Update
from pandasgui import show
from tqdm import tqdm
import seaborn as sns
from pprint import pprint
import pickle
import importlib

import src.execution
import src.model
import src.util
from contextlib import redirect_stdout

importlib.reload(src.execution)
importlib.reload(src.model)
importlib.reload(src.util)
from tabulate import tabulate
from src.data import NPY
from tensorflow.keras.models import load_model
from src.execution import CNNValidationExecution, WindowSplittingExecution, CNNTuningExecution, TrAdaBoostR2TrainingExecution, CNNTrainingExecution, TrAdaBoostR2ModelSetup_SingleModel_Execution
from src.model import CNN, build_model_for_TrAdaBoostR2, tradaBoostR2_setup
from src.experiment.multiple_cnn_singla_model_training import MultipleCNNTrainingOnExistingData
from src.experiment.multiple_trAdaBoostR2_trianings_specific_models import MultipleTrAdaBoostR2TrainingSpecificModleOutput
from src.experiment.evaluate_trAdaBoostR2_specific_models import EvaluateTrAdaBoostR2
from src.util import timestamp
import importlib

### TradaboostR2 Modells ###

TrAdaBoostR2 Modell mit drei Ausgängen auf den einzelnen Objekten aufbauen.

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
model_file = "assets\\models\\untrained\\synthetic-model\\syn-model.h5"
best_learning_rate=0.0005026453831067469
save_folder = os.path.join("build", "tradaboost_model", "training_30_TEST_02")
save_filename="training_09"

modell=tradaBoostR2_setup.TrAdaBoostR2ModelSetup(
    model_path=model_file,
    early_stoppping_model=True,
    patience_model=2,
    learningrate_model=best_learning_rate,
    n_estimators_tradaBoostR2=2,
    save_folder=save_folder,
    epochs_model=1,  # Wert bitte setzen
    batch_size_model=32,  # Wert bitte setzen
    early_stoppping_TraDaBoostR2=False,  # Wert bitte setzen
    learningrate_TraDaBoostR2=1  # Wert bitte setzen
)

training=TrAdaBoostR2TrainingExecution(tradaboostr2_model=modell,
                                                dateipfad_source_daten=dateipfad_femDaten,
                                                dateipfad_target_daten=dateipfad_realDaten,
                                                use_scaled_labels=True,
                                                model_file=model_file,
                                                save_folder=save_folder,
                                                save_filenssame="testmodel",
                                                training_dokumentation=True)


### Tuning der drei Modelle ###

- Modell mit drei Ausgängen verwenden bei denen jweils der input gleich ist. 
- Raum der Suche ist bei allen Modellen gleich, deswegen nur eine matadata.json Datei
- Verwendete Daten: Simulation
- Tuning erfolgt auf scalierten Labels


Ergebnis des Tuning:
- Phi: 3 Convolutional Layers
- X: 1 Convolutional Lyer
- Y: 1 Convolutional Lyer

In [15]:
x_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_X\\best-model.h5"
y_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_Y\\best-model.h5"
phi_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_Phi\\best-model.h5"
pfad_base_model="assets\\models\\untrained\\synthetic-model\\syn-model.h5"
model_paths=[x_modell,y_modell,phi_modell, pfad_base_model]
def summarize_model(filepath):
    model = load_model(filepath)
    summary = {
        "Model Name": os.path.basename(filepath),
        "Number of Layers": len(model.layers),
        "Trainable Params": model.count_params(),
        "Total Params": sum([layer.count_params() for layer in model.layers]),
    }
    return summary

# Alle Modelle zusammenfassen
summaries = [summarize_model(path) for path in model_paths]

# In DataFrame umwandeln und anzeigen
df = pd.DataFrame(summaries)
print(df.to_string(index=False))


   Model Name  Number of Layers  Trainable Params  Total Params
best-model.h5                 6              5274          5274
best-model.h5                11            121581        121581
best-model.h5                10              2119          2119
 syn-model.h5                 8              5002          5002


In [13]:
x_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_X\\best-model.h5"
y_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_Y\\best-model.h5"
phi_modell="assets\\models\\untrained\\seperated-models\\Verstellweg_Phi\\best-model.h5"
pfad_base_model="assets\\models\\untrained\\synthetic-model\\syn-model.h5"
model_paths=[x_modell,y_modell,phi_modell, pfad_base_model]
def print_model_summary(filepath):
    print(f"\n📦 Model: {os.path.basename(filepath)}")
    print(f"Path: {filepath}")

    model = load_model(filepath)
    
    # Zusammenfassung abfangen
    stream = io.StringIO()
    with redirect_stdout(stream):
        model.summary()
    summary_string = stream.getvalue()
    stream.close()
    print(summary_string)

for path in model_paths:
    print_model_summary(path)


print("Modell mit drie AUsgängen")


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10, 11)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 10, 15)         │           345 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 5, 15)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Verstellweg_X (Dense)           │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,276 (20.61 KB)

 Trainable params: 5,274 (20.60 KB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 5,004 (19.55 KB)

 Trainable params: 5,002 (19.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)


Modell mit drie AUsgängen


In [ ]:
SPLITTED_DATA_FOLDER = "build\\window_split\\sim_data_preprocessed\\1743966827\\"
CNNTuningExecution.execute_tuning_three_model(SPLITTED_DATA_FOLDER,train_on_scled_labels=True)

---
### Basline Modelle trainieren ###

- Modell getuned auf den SImulationsdaten
- Training auf den Realdaten und Auswertung auf den Realdaten
- Split 80/20
- 15 Datensätze verwenden, mit k-Fold

Fehler: Tuning auf scalierten Sim Daten, Training auf unskalierten Daten!!!

Daten erzeugen

In [ ]:
def validation_data_Mix(N: int, anteil):
    for seed in random.randint(0, 32000, N):
        DATA: str = "assets/data/real-data.csv"
        VALIDATION_SPLIT: bool = True # If the data should also be splitted into a validation set?
        TEST_SIZE: float = (100-anteil)/100
        BATCH_SPLIT: bool = True
        BATCHSIZE: int = 1800 # real-data: 1800, synthetic-data: 326
        SEED: int = seed # Seed for random state -> Split with same seed and data will always result in the same split
        INTERPOLATION: bool = False
        WINDOWSIZE: int = 10
        SEP: str = ";" # Separator of the csv file
        DECIMAL: str = "," # Decimal separator of the csv file
        
        WindowSplittingExecution.execute_with_scaler(DATA, 
                                        BATCH_SPLIT, 
                                        VALIDATION_SPLIT, 
                                        TEST_SIZE, 
                                        SEED, 
                                        BATCHSIZE, 
                                        INTERPOLATION, 
                                        WINDOWSIZE, 
                                        SEP, 
                                        DECIMAL)


In [ ]:
validation_data_Mix(N=15, anteil=80)
# Alle anderen Daten sind bereits erzeugt

## Training auf den Realdaten ##
Training auf den Datenanteilen von 5%, 10%, 15%, 20%, 50%, 80%
Siehe .py in den unterordnern

In [ ]:
#5% Realdaten
dateipfad_realDaten="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15"
dateipfad_modell="assets\\models\\untrained\\seperated-models"
ergebnis_ordner="build\\train_single_models\\training_1_Realdaten_5Prozent_N15"
MultipleCNNTrainingOnExistingData(target_folder=ergebnis_ordner,folder_data=dateipfad_realDaten, folder_base_model=dateipfad_modell, 
                                  result_folder_name_initial="training_1_SpecificModel")

In [ ]:
Model_Path="assets\\models\\untrained\\seperated-models"
dateipfad_realDaten="build\\window_split\\real-data\\1744062620\\"
CNNTrainingExecution.execute_three_models_training(model_folder=Model_Path, data_file=dateipfad_realDaten)

---
### TrAdaBoostR2 bei spezifischen Modellen für ein Output ###
- Tradaboost trainiert auf skalierten targets, nicht wie die single models, auf unskalierten Targets.
- ACHTUNG: Validierung des MAE auf den NICHT skalierten Daten, also dann in mm bzw rad oder grad, nciht sicher für Phi

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15\\"
model_file = "assets\\models\\untrained\\seperated-models"
save_folder = os.path.join("build", "tradaboost_model", "training_30_TEST_02")
save_filename="training_09"

modell=tradaBoostR2_setup.TrAdaBoostR2ModelSetup(
    model_path=model_file,
    early_stoppping_model=True,
    patience_model=2,
    learningrate_model=None,
    n_estimators_tradaBoostR2=2,
    save_folder=save_folder,
    epochs_model=1,  # Wert bitte setzen
    batch_size_model=32,  # Wert bitte setzen
    early_stoppping_TraDaBoostR2=False,  # Wert bitte setzen
    learningrate_TraDaBoostR2=1  # Wert bitte setzen
)

training=TrAdaBoostR2ModelSetup_SingleModel_Execution(tradaboostr2_model=modell,
                                                dateipfad_source_daten=dateipfad_femDaten,
                                                dateipfad_target_daten=dateipfad_realDaten,
                                                use_scaled_labels=True,
                                                model_file=model_file,
                                                save_folder=save_folder,
                                                save_filename="testmodel",
                                                training_dokumentation=False)


Debuggen TradaBoostR2Training

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed_offset_multiplitiv\\1744056915\\"
dateipfad_realDaten="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15\\"
                    "build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15"
model_file = "assets\\models\\untrained\\seperated-models"
save_folder = os.path.join("build", "tradaboost_model_specifiv_for_output", "trAdaBoostR2_1_Realdaten_5Prozent_N15")

MultipleTrAdaBoostR2TrainingSpecificModleOutput(target_data_folder=dateipfad_realDaten, 
                                                source_data_folder=dateipfad_femDaten,
                                                folder_base_model=model_file,
                                                result_folder=save_folder,
                                                result_folder_name_initial="trAdaBoostR2_1_SpecificModel")

In [ ]:
dateipfad_femDaten="build\\window_split\\sim_data_preprocessed\\1744139600\\"
dateipfad_realDaten ="build\\window_split\\real-data\\Realdaten_5Prozent_Random_N15\\"
model_file = os.path.join("assets", "models", "untrained", "seperated-models")
save_folder = os.path.join("build", "tradaboost_model_specifiv_for_output", "trAdaBoostR2_1_Realdaten_5Prozent_N15")
print("Starte Training mit:")
print(f"Source: {dateipfad_femDaten}")
print(f"Target: {dateipfad_realDaten}")
MultipleTrAdaBoostR2TrainingSpecificModleOutput(target_data_folder=dateipfad_realDaten, 
                                                source_data_folder=dateipfad_femDaten,
                                                folder_base_model=model_file,
                                                result_folder=save_folder,
                                                result_folder_name_initial="trAdaBoostR2_1_SpecificModel")

---
### Tradaboost Evaluieren ###

In [2]:
EvaluateTrAdaBoostR2.process_all_results(base_dir="build\\tradaboost_model_specifiv_for_output\\trAdaBoostR2_1_Realdaten_5Prozent_N15\\",
                                         use_scaled_labels=True)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 10, 11)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 10, 15)         │           345 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 5, 15)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 75)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Verstellweg_X (Dense)           │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,276 (20.61 KB)

 Trainable params: 5,274 (20.60 KB)

 Non-trainable params: 0 (0.00 B)

...

 Trainable params: 2,119 (8.28 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None
3804/3804 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step
build\window_split\real-data\Realdaten_5Prozent_Random_N15\1745867781\scalers_labels.pkl
preds_full.shape: (121720, 3)
y_test.shape: (121720, 0, 3)


ValueError: cannot reshape array of size 0 into shape (121720,3)